# Examples
## mutlimethod
Multimethods are a mapping of signatures (tuple of types) to functions.  They maintain an efficient dispatch tree, and cache the called signatures.

In [ ]:
from multimethod import multimethod
import operator

classic_div = multimethod(operator.truediv)
classic_div[int, int] = operator.floordiv
classic_div

In [ ]:
classic_div(3, 2)

In [ ]:
classic_div(3.0, 2)

In [ ]:
classic_div

Multimethods introspect type annotations and use the name to find existing multimethods.

In [ ]:
import itertools
from collections.abc import Iterable, Sequence


@multimethod
def chunks(values: Iterable, size):
    it = iter(values)
    return iter(lambda: list(itertools.islice(it, size)), [])


@multimethod
def chunks(values: Sequence, size):
    for index in range(0, len(values), size):
        yield values[index : index + size]


list(chunks(iter('abcde'), 3))

In [ ]:
list(chunks('abcde', 3))

Multimethods also have an explicit `register` method similar to `functools.singledispatch`.

In [ ]:
@multimethod
def window(values, size=2):
    its = itertools.tee(values, size)
    return zip(*(itertools.islice(it, index, None) for index, it in enumerate(its)))


@window.register
def _(values: Sequence, size=2):
    for index in range(len(values) - size + 1):
        yield values[index : index + size]


list(window(iter('abcde')))

In [ ]:
list(window('abcde'))

## overload
Whereas multimethods require an `issubclass` relationship, overloads dispatch on any predicates.

In [ ]:
import asyncio
import time
from concurrent import futures
from multimethod import overload


@overload
def wait(timeout, func, *args):
    return futures.ThreadPoolExecutor().submit(func, *args).result(timeout)


@overload
async def wait(timeout, func: asyncio.iscoroutinefunction, *args):
    return await asyncio.wait_for(func(*args), timeout)


wait(0.5, time.sleep, 0.01)

In [ ]:
wait(0.5, asyncio.sleep, 0.01)

## typing subscripts
Support for type hints with subscripts.

In [ ]:
import bisect
import random


@multimethod
def samples(weights: dict):
    """Generate weighted random samples using bisection."""
    keys = list(weights)
    totals = list(itertools.accumulate(weights.values()))
    values = [total / totals[-1] for total in totals]
    while True:
        yield keys[bisect.bisect_right(values, random.random())]


@multimethod
def samples(weights: dict[object, int]):
    """Generate weighted random samples more efficiently."""
    keys = list(itertools.chain.from_iterable([key] * weights[key] for key in weights))
    while True:
        yield random.choice(keys)


weights = {'a': 1, 'b': 2, 'c': 3}
next(samples(weights))

In [ ]:
weights = {'a': 1.0, 'b': 2.0, 'c': 3.0}
next(samples(weights))